In [ ]:
a=[1,2,3]
b=a+[8]

In [7]:
!pip install -U scikit-image


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skimage.segmentation import slic, find_boundaries
from skimage.io import imread
from skimage.color import label2rgb
from scipy.ndimage import center_of_mass
from torch_geometric.data import Data
import argparse
import os

# Load image from path
def load_image_s(image_path):
    return imread(image_path)

# Generate the superpixel graph
def superpixel_graph(image_path, label, n_segments=50, compactness=10):
    img = load_image_s(image_path)
    labels = slic(img, n_segments=n_segments, compactness=compactness, start_label=0)
    num_segments = labels.max() + 1

    # Compute node features (average color per superpixel)
    avg_colors = np.array([img[labels == i].mean(axis=0) for i in range(num_segments)])
    x = torch.tensor(avg_colors, dtype=torch.float)

    # Manually find adjacency
    adjacency = set()
    for y in range(labels.shape[0] - 1):
        for x_ in range(labels.shape[1] - 1):
            current_label = labels[y, x_]
            neighbors = [labels[y + 1, x_], labels[y, x_ + 1]]
            for neighbor_label in neighbors:
                if current_label != neighbor_label:
                    edge = tuple(sorted([current_label, neighbor_label]))
                    adjacency.add(edge)

    edge_index = torch.tensor(list(adjacency), dtype=torch.long).T

    # Edge attributes: color difference
    edge_attr = []
    for n1, n2 in adjacency:
        color_diff = np.linalg.norm(avg_colors[n1] - avg_colors[n2])
        edge_attr.append([color_diff])
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=torch.tensor([label])), labels, img

# Plot the image with graph overlay
def show_graph_on_image(image_path, label):
    data, labels, img = superpixel_graph(image_path, label)

    # Compute centroids of superpixels
    centroids = center_of_mass(np.ones_like(labels), labels, range(labels.max() + 1))
    centroids = np.array(centroids)

    # Plot image with boundaries and graph
    plt.figure(figsize=(10, 10))
    plt.imshow(label2rgb(labels, img, kind='avg'))

    # Plot edges
    for i in range(data.edge_index.shape[1]):
        n1, n2 = data.edge_index[:, i]
        y1, x1 = centroids[n1]
        y2, x2 = centroids[n2]
        plt.plot([x1, x2], [y1, y2], 'r-', linewidth=1)

    # Plot nodes
    for y, x in centroids:
        plt.plot(x, y, 'bo', markersize=4)

    plt.title("Superpixel Graph Overlay")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Command-line interface
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Display superpixel graph on image")
    parser.add_argument("image_path", default=r"D:\OBM\CODES\HeathlandSpeciesClassifier\dataset\images\lidl\calluna\im31_63_.1.0.jpg", type=str, help="Path to the input image")
    parser.add_argument("--label", type=int, default=0, help="Optional label for the image")
    args = parser.parse_args()

    if not os.path.exists(args.image_path):
        raise FileNotFoundError(f"Image not found: {args.image_path}")

    show_graph_on_image(args.image_path, args.label)


In [12]:
from skimage.segmentation import slic
print("SLIC import successful")


ImportError: cannot import name 'slic' from 'skimage.segmentation' (unknown location)

In [13]:

!pip uninstall -y scikit-image


Found existing installation: scikit-image 0.19.3
Uninstalling scikit-image-0.19.3:
  Successfully uninstalled scikit-image-0.19.3


In [14]:
pip install --no-cache-dir scikit-image


   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------- ----------------------- 5.2/12.8 MB 35.3 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 53.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float

# Load an image (change the path as needed)
image_path = 'your_image.jpg'  # Replace with a real path
image = img_as_float(imread(image_path))

# Apply SLIC superpixels
segments = slic(image, n_segments=100, compactness=10, start_label=0)

# Show the image with boundaries
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(mark_boundaries(image, segments))
ax.set_title('SLIC Superpixels')
ax.axis('off')
plt.tight_layout()
plt.show()


None
